In [1]:
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, fbeta_score, precision_score, recall_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, fbeta_score, precision_score, recall_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import glob
# import wandb
# from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
%matplotlib inline

2024-02-21 11:40:04.405006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 11:40:04.405099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 11:40:04.563907: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
image_path = "/kaggle/input/bacteria-figshare/"
# tmp_data_path = "/content/tmp_data/"
root_dir = '/kaggle/working/data/'

In [3]:
# INPUT_SIZE = 71
# input_shape = (INPUT_SIZE, INPUT_SIZE, 3)  # Assuming color images (3 channels)
# num_classes = 24
# batch_size = 32

# def pad_array(input_array, target_shape=(224, 224, 3)):
#     # Get the current shape of the input array
#     current_shape = input_array.shape

#     # Calculate the difference between the target shape and current shape
#     pad_width = [(0, max(0, target_shape[i] - current_shape[i])) for i in range(len(target_shape))]

#     # Pad the array using numpy.pad
#     padded_array = np.pad(input_array, pad_width, mode='constant', constant_values=0)

#     return padded_array

# import cv2
# import os
# import glob
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from keras.preprocessing.image import ImageDataGenerator, array_to_img
# import numpy as np
# from skimage.transform import resize
# from PIL import Image

# def generate_images_from_and_to_given_path(image_path, output_path, num_augmented_images, MAX_SIZE):
#     INPUT_SIZE = MAX_SIZE
   
#     datagen = ImageDataGenerator(
#     rotation_range=30,
#     width_shift_range=0.05,
#     height_shift_range=0.05,
#     shear_range=0.2,
#     zoom_range=0.3,
#     horizontal_flip=True,
#     brightness_range=[0.5, 1.5]
#     )
   
#     for annotated_file in glob.glob(os.path.join(f'{image_path}annot_YOLO/', '*.txt')):
#         with open(annotated_file) as f:        
#             tmp_list = []
#             for text in f:
#                 second_tmp = []
#                 for i, coordinates in enumerate(text.split(' ')):
#                     if i == 4:
#                         second_tmp.append(coordinates.strip())
#                     else:
#                         second_tmp.append(coordinates)
#                 tmp_list.append(second_tmp)

#             # For image cropping
#             image = cv2.imread(f'{image_path}{annotated_file[-14:-4]}.jpg')
#             boxes = tmp_list

#             shift = 0.01
#             for i, box in enumerate(boxes):
#                 x_min = float(box[1])
#                 y_min = float(box[2])
#                 x_min = float(box[1])
#                 y_min = float(box[2])
#                 width = float(box[3]) + 0.014
#                 height = float(box[4])+ 0.014

#                 x_max = x_min + width
#                 y_max = y_min + height
#                 cropped = image[int(y_min*image.shape[0]):int(y_max*image.shape[0]),
#                               int(x_min*image.shape[1]):int(x_max*image.shape[1])]

#                 if cropped.size == 0:
#                     print("Locha: ", annotated_file)
#                 else:
#                     if cropped.shape[0] > MAX_SIZE or cropped.shape[1] > MAX_SIZE:
#                         cropped = cv2.resize(cropped, (MAX_SIZE, MAX_SIZE))

#                     # Generate and plot augmented images
#                     cropped_batch = np.expand_dims(cropped, axis=0)  # Add batch dimension
#                     generate_and_save_image_to_given_path(output_path+"", cropped_batch, annotated_file, datagen, num_augmented_images, INPUT_SIZE)

# def generate_and_save_image_to_given_path(output_path, cropped_batch, annotated_file, datagen, num_augmented_images, INPUT_SIZE):
#     count =0
#     # for getting colonies coordinates
#     if not os.path.exists(f'{output_path}{annotated_file[-14:-10]}'):
#         os.makedirs(f'{output_path}{annotated_file[-14:-10]}')
   
#     for i, augmented_img in enumerate(datagen.flow(cropped_batch, batch_size=1)):
#         augmented_img = pad_array(augmented_img[0], target_shape=(INPUT_SIZE, INPUT_SIZE, 3))

# #         augmented_img = cv2.cvtColor(augmented_img, cv2.COLOR_BGR2RGB)
#         augmented_img = np.array(augmented_img).astype(np.uint8)
       
# #         min_val,max_val=augmented_img.min(),augmented_img.max()
# #         augmented_img = 255.0*(augmented_img - min_val)/(max_val - min_val)
# #         augmented_img = augmented_img.astype(np.uint8)
       
#         image = Image.fromarray(augmented_img, 'RGB')
#         image.save(f'{output_path}{annotated_file[-14:-10]}/{count}_{annotated_file[-14:-4]}.jpg', bbox_inches='tight', pad_inches=0)
        
# #         plt.imshow(cv2.cvtColor(augmented_img, cv2.COLOR_BGR2RGB))
# #         plt.axis('off')  # Turn off axis
# #         plt.savefig(f'{output_path}{annotated_file[-14:-10]}/{count}_{annotated_file[-14:-4]}.jpg', bbox_inches='tight', pad_inches=0)  # Save the image
# #         plt.show()
       
#         count+=1

#         if i == num_augmented_images - 1:
#             break
           
           
# generate_images_from_and_to_given_path("/kaggle/input/bacteria-figshare/", "/kaggle/working/augmented_dataset2/", num_augmented_images=15, MAX_SIZE=71)

In [4]:
INPUT_SIZE = 75
input_shape = (INPUT_SIZE, INPUT_SIZE, 3)  # Assuming color images (3 channels)
num_classes = 24
batch_size = 32

In [5]:
def pad_array(input_array, target_shape=(224, 224, 3)):
    # Get the current shape of the input array
    current_shape = input_array.shape

    # Calculate the difference between the target shape and current shape
    pad_width = [(0, max(0, target_shape[i] - current_shape[i])) for i in range(len(target_shape))]

    # Pad the array using numpy.pad
    padded_array = np.pad(input_array, pad_width, mode='constant', constant_values=0)

    return padded_array

In [6]:
for annotated_file in glob.glob(os.path.join('/kaggle/input/bacteria-figshare/annot_YOLO/', '*.txt')):
    with open(annotated_file) as f:
        # for getting colonies coordinates
        if os.path.exists(f'/kaggle/working/data/{annotated_file[-14:-10]}') == False:
            os.makedirs(f'/kaggle/working/data/{annotated_file[-14:-10]}')
        tmp_list = []
        for text in f:
            second_tmp = []
            for i, coordinates in enumerate(text.split(' ')):
                if i == 4:
                    second_tmp.append(coordinates.strip())
                else:
                    second_tmp.append(coordinates)
            tmp_list.append(second_tmp)

        # For image croping
        image = cv2.imread(f'{image_path}{annotated_file[-14:-4]}.jpg')
        boxes = tmp_list

        shift = 0.01

        for i, box in enumerate(boxes):
            x_min = float(box[1]) - shift
            y_min = float(box[2]) - shift
            width = float(box[3])
            height = float(box[4])

            x_max = x_min + width
            y_max = y_min + height

            cropped = image[int(y_min*image.shape[0]):int(y_max*image.shape[0]),
                          int(x_min*image.shape[1]):int(x_max*image.shape[1])]

            cropped = pad_array(cropped, target_shape=(INPUT_SIZE, INPUT_SIZE, 3))

            save_path = f'/kaggle/working/data/{annotated_file[-14:-10]}/'+str(i)+f'_{annotated_file[-14:-4]}.jpg'
            if(cropped.size==0):
                print("Locha: ", annotated_file)
            else:
                cv2.imwrite(save_path, cropped)

In [7]:
# import joblib
# model = joblib.load('/kaggle/input/densnet/keras/ntohing/1/XceptionNet.pkl')

In [8]:
root_dir = '/kaggle/working/data'
datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     vertical_flip=True,
#     fill_mode='nearest',
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    root_dir,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    root_dir,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical',
    subset='validation'
)


Found 45502 images belonging to 24 classes.
Found 11363 images belonging to 24 classes.


In [9]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
# from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall

batch_size = 32
# Create Xception model
base_model = keras.applications.EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)

# for layer in base_model.layers:
#     layer.trainable = False

    
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='sigmoid'))
model.add(Dense(1024, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='sigmoid'))
model.add(Dropout(0.5))  # Dropout layer for regularization
model.add(Dense(num_classes, activation='softmax'))



# # Compile the model
model.compile(optimizer=Nadam(weight_decay=0.00001), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Data Augmentation

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)



31790344/31790344 [==============================] - 0s 0us/step


In [10]:
# # Train the model
epochs = 20
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks = [reduce_lr]
)

Epoch 1/20


2024-02-21 11:42:15.617407: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb2/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1708515745.560833     114 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1421/1421 [==============================] - 281s 123ms/step - loss: 0.7602 - accuracy: 0.7786 - precision: 0.8598 - recall: 0.7228 - val_loss: 0.4045 - val_accuracy: 0.8835 - val_precision: 0.8973 - val_recall: 0.8727 - lr: 0.0010
Epoch 2/20
1421/1421 [==============================] - 169s 119ms/step - loss: 0.3285 - accuracy: 0.9012 - precision: 0.9185 - recall: 0.8858 - val_loss: 0.2328 - val_accuracy: 0.9329 - val_precision: 0.9423 - val_recall: 0.9268 - lr: 0.0010
Epoch 3/20
1421/1421 [==============================] - 169s 119ms/step - loss: 0.2603 - accuracy: 0.9214 - precision: 0.9345 - recall: 0.9113 - val_loss: 0.2448 - val_accuracy: 0.9243 - val_precision: 0.9329 - val_recall: 0.9198 - lr: 0.0010
Epoch 4/20
1421/1421 [==============================] - 168s 118ms/step - loss: 0.2257 - accuracy: 0.9310 - precision: 0.9430 - recall: 0.9220 - val_loss: 0.2400 - val_accuracy: 0.9279 - val_precision: 0.9394 - val_recall: 0.9210 - lr: 0.0010
Epoch 5/20
1421/1421 [=================

In [11]:
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, top_k_accuracy_score, confusion_matrix, precision_score
import numpy as np

# Assuming you already have the trained model and validation data generator (val_generator) from the previous code

# Evaluate the model on the validation set
predictions = model.predict(val_generator)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Convert true labels to class labels
true_labels = val_generator.classes

# Calculate top-3 and top-4 accuracy
top3_accuracy = top_k_accuracy_score(true_labels, predictions, k=3)
top5_accuracy = top_k_accuracy_score(true_labels, predictions, k=5)

# Calculate overall accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate recall, F1 score, and classification report
recall = recall_score(true_labels, predicted_labels, average='weighted')
precision = precision_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')
conf_matrix = confusion_matrix(true_labels, predicted_labels)
classification_rep = classification_report(true_labels, predicted_labels)

# Print the metrics
print(f'Top-3 Accuracy: {top3_accuracy * 100:.2f}%')
print(f'Top-5 Accuracy: {top5_accuracy * 100:.2f}%')
print(f'Overall Accuracy: {overall_accuracy * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'F1 Score: {f1 * 100:.2f}%')
print(conf_matrix)
print('Classification Report:')
print(classification_rep)


356/356 [==============================] - 10s 22ms/step
Top-3 Accuracy: 99.59%
Top-5 Accuracy: 99.78%
Overall Accuracy: 97.34%
Recall: 97.34%
Precision: 97.37%
F1 Score: 97.34%
[[  75    0    0    0    1    0    0    0    0    0    0    0    0    0
     0    0    0    1    0    0    1    1    0    0]
 [   0  304    0    0    1    0    0    0    0    0    0    0    0    0
     0    1    0    0    0    0    0    0    0    0]
 [   1    1   53    0    3    0    0    0    0    0    0    0    0    0
     0    1    0    0    0    0    0    0    0    0]
 [   0    0    0   59    1    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    1  816    0    0    0    0    0    0    0    0    0
     1    0    0    1    0    0    1    0    0    0]
 [   0    0    0    0    0 1049    1    0    0    8    0    0    0    1
     0    0    0    0    9    0    0    2   31    1]
 [   0    0    0    0    2    0  204    0    7    1    0    0    0    0


In [ ]:
model.save('EfficientNetB1.pkl')

In [ ]:
import seaborn as sns
def plot_confusion_matrix(y_true, y_pred, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    """
    Plot the confusion matrix for multi-class classification.

    Parameters:
        y_true (array-like): True labels.
        y_pred (array-like): Predicted labels.
        classes (list): List of class names.
        title (str): Title of the plot.
        cmap (matplotlib.colors.Colormap): Colormap for the plot.
    """
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(len(classes)+2, len(classes)+2))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=True, fmt="d", cmap=cmap, linewidths=.5, square=True,
                xticklabels=classes, yticklabels=classes)

    plt.title(title)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

# Example usage:
# Replace y_true and y_pred with your actual and predicted labels

plot_confusion_matrix(true_labels, predicted_labels, classes=["sp01", "sp02", "sp03", "sp04", "sp05", "sp06", "sp07", "sp08", "sp09", "sp10", "sp11", "sp12", "sp13", "sp14", "sp15", "sp16", "sp17", "sp18", "sp19", "sp20", "sp21", "sp22", "sp23"])


In [ ]:
from tensorflow.keras.preprocessing import image

# Preprocess the image
img_path = '/kaggle/working/data/sp21/299_sp21_img51.jpg'
img = image.load_img(img_path, target_size=(75,75,3))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

model.predict(img_array).argmax()+1